In [ ]:
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import json
import subprocess
from pathlib import Path

import torch

In [ ]:
upper_lim_pix_val = np.load('upper_lim_pix_val.npy')
lower_lim_pix_val = np.load('lower_lim_pix_val.npy')

In [ ]:
def eight_bit_scaler(x, axis=None):
  min = x.min(axis=axis, keepdims=True)
  max = x.max(axis=axis, keepdims=True)
  min_max_scaled = (x-min)/(max-min)
  pre_eight_bit_scaled = 255 * min_max_scaled
  result = pre_eight_bit_scaled.astype(np.uint8)
  return result

def averaged_adaptive_threshold(gamma_corrected_8bit, window_size_list, C=-7):
  img_arr = np.zeros((len(window_size_list), gamma_corrected_8bit.shape[0], gamma_corrected_8bit.shape[1]))
  
  for i, window_size in enumerate(window_size_list):
    binarized = cv2.adaptiveThreshold(gamma_corrected_8bit, 
                                       255, 
                                       cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                       cv2.THRESH_BINARY, 
                                       window_size, 
                                       C)
    img_arr[i, :, :] = binarized

    # result
    # plt.figure(figsize=(12, 8))
    # plt.subplot(1, 2, 1)
    # plt.imshow(gamma_corrected_8bit,
    #           interpolation = 'nearest', 
    #           cmap = 'viridis')
    # plt.title('Before')
    # plt.subplot(1, 2, 2)
    # plt.imshow(binarized,
    #           interpolation = 'nearest', 
    #           cmap = 'viridis')
    # plt.title('Window size = ' + str(window_size))
    # plt.show()

  return np.mean(img_arr, axis=0)

def preprocessing(org_img, 
                  upper_lim_pix_val, lower_lim_pix_val, 
                  flip_flag=False, 
                  clipLimit=2.0, tileGridSize=(8, 8), 
                  blur_kernel_size=(3, 3), 
                  bilateral_d=3, bilateral_sigmaColor=200, bilateral_sigmaSpace=50, 
                  sharp_kernel_value=1.0, 
                  inflate_flag=True,
                  gamma = 4.8, 
                  window_size_list=[11, 21, 31, 41, 51, 61, 71],
                  C=-7, 
                  ensemble_flag=True):

  # flip the values
  if flip_flag:
    flipped_img = cv2.bitwise_not(org_img)
  else:
    flipped_img = org_img

  # normalize
  temp = np.where(flipped_img > upper_lim_pix_val, upper_lim_pix_val, flipped_img)
  normed_img = np.where(temp < lower_lim_pix_val, lower_lim_pix_val, temp)

  # contrast
  # clipLimit: If you want to enhance edges, you should raise the value
  normed_img_8 = np.array(normed_img, dtype=np.uint8)
  clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
  cl1 = clahe.apply(normed_img_8)

#   plt.figure(figsize=(10, 6))

#   plt.subplot(1, 2, 1)
#   plt.imshow(normed_img_8,
#             interpolation = 'nearest', 
#             cmap = 'gray')
#   plt.title('Before')

#   plt.subplot(1, 2, 2)
#   plt.imshow(cl1,
#             interpolation = 'nearest', 
#             cmap = 'gray')
#   plt.title('After')

#   plt.suptitle('Contrast', fontsize=10)
#   plt.show()

  # blur
  blured_img = cv2.blur(cl1, blur_kernel_size)

#   plt.figure(figsize=(10, 6))

#   plt.subplot(1, 2, 1)
#   plt.imshow(cl1,
#             interpolation = 'nearest', 
#             cmap = 'gray')
#   plt.title('Before')

#   plt.subplot(1, 2, 2)
#   plt.imshow(blured_img,
#             interpolation = 'nearest', 
#             cmap = 'gray')
#   plt.title('After')

#   plt.suptitle('Blur', fontsize=10)
#   plt.show()

  # bilateral filter
  # bilateral_sigmaColor: If bilateral_sigmaColor is large, a large weight is adopted even if the difference in pixel values ​​is large.
  # bilateral_sigmaSpace: If bilateral_sigmaSpace is large, a large weight is adopted even if the distance between pixels is wide.
  bilateral_filtered_img = cv2.bilateralFilter(blured_img.astype(np.float32), bilateral_d, bilateral_sigmaColor, bilateral_sigmaSpace)

#   plt.figure(figsize=(10, 6))

#   plt.subplot(1, 2, 1)
#   plt.imshow(blured_img,
#             interpolation = 'nearest', 
#             cmap = 'gray')
#   plt.title('Before')

#   plt.subplot(1, 2, 2)
#   plt.imshow(bilateral_filtered_img,
#             interpolation = 'nearest', 
#             cmap = 'gray')
#   plt.title('After')

#   plt.suptitle('Bilateral', fontsize=10)
#   plt.show()

  # Sharpening
  # sharp_kernel_value: If you want to enhance edges, you should raise the value
  sharpening_kernel = np.array([[-sharp_kernel_value, -sharp_kernel_value, -sharp_kernel_value], 
                                [-sharp_kernel_value, 1+8*sharp_kernel_value, -sharp_kernel_value], 
                                [-sharp_kernel_value, -sharp_kernel_value, -sharp_kernel_value]])
  sharpen = cv2.filter2D(bilateral_filtered_img, ddepth=-1, kernel=sharpening_kernel)
  # plt.imshow(sharpen, cmap = "gray")
  # plt.title('Sharpening')
  # plt.show()
  
  # Inflate the edge
  if inflate_flag:
    ## Erosion
    kernel = np.ones((5,5), np.uint8)
    erosion = cv2.erode(sharpen, kernel, iterations = 2)
    # plt.imshow(erosion, cmap = "gray")
    # plt.title('Erosion')
    # plt.show()

    ## Dilation
    kernel = np.ones((4,4), np.uint8)
    dilation = cv2.dilate(erosion, kernel, iterations = 2)
    # plt.imshow(dilation, cmap = "gray")
    # plt.title('Dilation')
    # plt.show()
  else:
    dilation = sharpen

  #　Gamma correction
  # gamma: If you want to enhance edges, you should raise the value.
  #          However, modules may be separated due to clustering of pixel values (gamma correction.)
  dilation_8bit = eight_bit_scaler(dilation)
  gamma_corrected = np.power(dilation_8bit / float(np.max(dilation_8bit)), gamma)
  gamma_corrected_8bit = eight_bit_scaler(gamma_corrected)
#   plt.figure(figsize=(10, 6))
#   plt.subplot(1, 2, 1)
#   plt.imshow(dilation_8bit,
#             interpolation = 'nearest', 
#             cmap = 'gray')
#   plt.title('Before')
#   plt.subplot(1, 2, 2)
#   plt.imshow(gamma_corrected_8bit,
#             interpolation = 'nearest', 
#             cmap = 'gray')
#   plt.title('After')
  
#   plt.suptitle('Gamma correction', fontsize=10)
#   plt.show()

  # Binarized
  ave = averaged_adaptive_threshold(gamma_corrected_8bit, window_size_list, C=-7)

  # plt.figure()
  # plt.imshow(ave,
  #           interpolation = 'nearest', 
  #           cmap = 'gray')
  # plt.colorbar()
  # plt.title('AVE')
  # plt.show()

  if ensemble_flag:
    ave_bi = np.where(ave > 255/2, 255, 0)

    # plt.figure()
    # plt.imshow(ave_bi,
    #           interpolation = 'nearest', 
    #           cmap = 'gray')
    # plt.title('bi')
    # plt.show()
  else:
    ave_bi = np.where(ave > 0, 255, 0)

    # plt.figure()
    # plt.imshow(ave_bi,
    #           interpolation = 'nearest', 
    #           cmap = 'gray')
    # plt.title('bi')
    # plt.show()

  return ave_bi

def select_contours(img_binarized, 
                    area_min=50, area_max=3000, 
                    aspect_min=0.5, aspect_max=1.0):
  # 輪郭の検出
  contours, hierarchy = cv2.findContours(img_binarized.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  im_con = cv2.drawContours(np.zeros_like(img_binarized.astype(np.uint8)), contours, -1, 255, -1)
  # plt.figure(figsize=(10, 5))
  # plt.subplot(1, 1, 1)
  # plt.imshow(im_con, cmap='gray')
  # plt.title('Contours')
  # plt.show()

  panel_contours = []

  # 輪郭を１つずつ書き込んで出力
  for i in range(len(contours)):
      area = cv2.contourArea(contours[i])
      if area_min <= area <= area_max:
        # peri = cv2.arcLength(contours[i], True)
        # squareness = 4 * np.pi * area / peri**2
        # if 0.3 <= squareness <= 0.8: # https://answers.opencv.org/question/171583/eliminate-unwanted-contours-opencv/
        rect = cv2.minAreaRect(contours[i])

        (x, y), (width, height), angle = rect
        aspect_ratio = min(width, height) / max(width, height)

        if aspect_min <= aspect_ratio <= aspect_max:
          box = cv2.boxPoints(rect)
          box = np.int0(box)
          
          if cv2.contourArea(box) <= area_max:
            panel_contours.append(box)

  return panel_contours

In [ ]:
class ExifToolWraper(object):

    def __init__(self, exiftool_path="/usr/local/bin/exiftool", is_debug=False):
        self.exiftool_path = exiftool_path
        self.is_debug = is_debug


    def get_meta(self, img_filename):

        if not os.path.isfile(img_filename):
            raise ValueError("Input file does not exist or this user don't have permission on this file")

        # meta_json = subprocess.check_output(
        #     [self.exiftool_path, '-Model', '-AbsoluteAltitude', '-RelativeAltitude', '-GimbalRollDegree', '-GimbalYawDegree', '-GimbalPitchDegree', '-j', img_filename])
        # meta_json = subprocess.check_output([self.exiftool_path, '-j', img_filename])
        # meta_json = subprocess.check_output([self.exiftool_path, '-j', str(img_filename).replace(' ', '\ ')])
        meta_json = subprocess.check_output([self.exiftool_path, '-j', str(img_filename)])
        meta = json.loads(meta_json.decode())[0]

        return meta

In [ ]:
exiftool = ExifToolWraper( exiftool_path="/usr/local/bin/exiftool")

In [ ]:
processed_folder = Path('data/processed')
pred_dir = processed_folder / 'predictions_test'
if not pred_dir.exists(): pred_dir.mkdir()
cnt = 0

In [ ]:
"ENEOSうるま市ソーラー発電所_2021-08-11 11_02_15_サーモ" in file

True

In [ ]:
for folder in glob.glob("/Users/y.shiraishi/Documents/drone/module_detection/data/*"):
    ins_f = lambda x:x in folder
    if any(map(ins_f, ("ENEOSうるま市ソーラー発電所_2021-08-11 11_02_15_サーモ", 
                       "七飯峠下太陽光発電所_2021-09-17 13_32_34_サーモ", 
                       'こんぴら_2021-07-20 10_17_46_サーモ', 
                       '第一吹上浜ソーラーエナジー_2021-07-28 09_09_27_サーモ'))):
        for subfolder in glob.glob(folder+'/*'):
            print(subfolder)
            for file in  glob.glob(subfolder+'/*'):
                file_name = os.path.split(file)[-1]
                print(file_name)

                if "ENEOSうるま市ソーラー発電所_2021-08-11 11_02_15_サーモ" in file:
                    flip_flag = False
                    clipLimit = 2.0
                    tileGridSize = 8
                    blur_kernel_size = 3
                    bilateral_d = 3
                    bilateral_sigmaColor = 200
                    bilateral_sigmaSpace = 50
                    sharp_kernel_value = 10.0
                    inflate_flag = True
                    gamma = 3
                    window_size_list = [51, 61, 71]
                    C = -7
                    ensemble_flag = True

                    area_min=50
                    area_max=300
                    aspect_min=0.3
                    aspect_max=0.9
                elif "七飯峠下太陽光発電所_2021-09-17 13_32_34_サーモ" in file:
                    flip_flag = False
                    clipLimit = 2.0
                    tileGridSize = 8
                    blur_kernel_size = 3
                    bilateral_d = 3
                    bilateral_sigmaColor = 200
                    bilateral_sigmaSpace = 50
                    sharp_kernel_value = 50.0
                    inflate_flag = True
                    gamma = 4.8
                    window_size_list = [51, 61, 71]
                    C = -7
                    ensemble_flag = True
                    area_min=150
                    area_max=550
                    aspect_min=0.2
                    aspect_max=0.9
                elif "こんぴら_2021-07-20 10_17_46_サーモ" in file:
                    flip_flag = False
                    clipLimit = 2.0
                    tileGridSize = 8
                    blur_kernel_size = 3
                    bilateral_d = 3
                    bilateral_sigmaColor = 200
                    bilateral_sigmaSpace = 50
                    sharp_kernel_value = 10.0
                    inflate_flag = True
                    gamma = 8.0
                    window_size_list = [31, 41, 51]
                    C = -7
                    ensemble_flag = True
                    area_min=80
                    area_max=300
                    aspect_min=0.3
                    aspect_max=0.6
                elif "第一吹上浜ソーラーエナジー_2021-07-28 09_09_27_サーモ" in file:
                    flip_flag = False
                    clipLimit = 2.0
                    tileGridSize = 8
                    blur_kernel_size = 3
                    bilateral_d = 3
                    bilateral_sigmaColor = 200
                    bilateral_sigmaSpace = 50
                    sharp_kernel_value = 50.0
                    inflate_flag = True
                    gamma = 4.8
                    window_size_list = [21, 31, 41, 51, 61, 71]
                    C = -7
                    ensemble_flag = True
                    area_min=80
                    area_max=300
                    aspect_min=0.3
                    aspect_max=0.6

                img = Image.open(file)

                original = np.expand_dims(np.asarray(img).transpose([2, 0, 1]), axis=0)

                x = torch.as_tensor(original, dtype=torch.float32)

                if 'solar' in subfolder:
                    patches = x.unfold(2, 224, 128).unfold(3, 224, 128)
                else:
                    patches = x.unfold(2, 224, 64).unfold(3, 224, 64)

                cnt = 1
                for i in range(patches.shape[2]):
                    for j in range(patches.shape[3]):
                        cropped = patches[0, : , i, j, :, :].detach().numpy().copy()
                        gray_img = cv2.cvtColor(cropped.transpose([2, 1, 0]), cv2.COLOR_RGB2GRAY).astype(np.uint8)

                        img_processed = preprocessing(gray_img, 
                                                      upper_lim_pix_val, lower_lim_pix_val, 
                                                      flip_flag = flip_flag, 
                                                      clipLimit = clipLimit, tileGridSize = (tileGridSize, tileGridSize), 
                                                      blur_kernel_size = (blur_kernel_size, blur_kernel_size), 
                                                      bilateral_d = bilateral_d, bilateral_sigmaColor = bilateral_sigmaColor, bilateral_sigmaSpace = bilateral_sigmaSpace, 
                                                      sharp_kernel_value = sharp_kernel_value, 
                                                      inflate_flag = inflate_flag, 
                                                      gamma = gamma, 
                                                      window_size_list = window_size_list, 
                                                      C = C,  
                                                      ensemble_flag = ensemble_flag)

                        modules_contours = select_contours(img_processed, 
                                                           area_min=area_min, area_max=area_max, 
                                                           aspect_min=aspect_min, aspect_max=aspect_max)

                        im_con = cv2.drawContours(np.zeros_like(gray_img), modules_contours, -1, 255, -1)
                        mask = cv2.bitwise_and(gray_img, im_con)
                        
#                         clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
#                         cl1 = clahe.apply(gray_img)

#                         denoise = cv2.fastNlMeansDenoising(cl1, None, templateWindowSize=7, searchWindowSize=21, h=30)
                        denoise = cv2.fastNlMeansDenoising(gray_img, None, templateWindowSize=7, searchWindowSize=21, h=30)

                        ret2, img_otsu = cv2.threshold(denoise, 0, 255, cv2.THRESH_OTSU)

                        double_mask = cv2.bitwise_and(mask, img_otsu)

                        ret2, final_mask = cv2.threshold(double_mask, 0, 255, cv2.THRESH_OTSU)

                        # pred_name = file_name + f"_{cnt}_shiraishi_pred.png"
                        # cv2.imwrite(str(pred_dir / os.path.split(current)[-1] / pred_name), final_mask)

                        # for train data
                        mask_arr = (final_mask/255).astype(np.float64)

                        if 'solar' in subfolder:
                            arr_name = os.path.split(folder)[-1] + '_' + file_name + f"_{cnt}"
                            np.save('/Users/y.shiraishi/Documents/github/solar-panel-segmentation/data/processed/20211128_thermal/solar/mask/' + arr_name, mask_arr)
                            np.save('/Users/y.shiraishi/Documents/github/solar-panel-segmentation/data/processed/20211128_thermal/solar/org/' + arr_name, 
                                    np.concatenate([np.expand_dims(gray_img, 0), np.expand_dims(gray_img, 0), np.expand_dims(gray_img, 0)]))
                            cv2.imwrite('/Users/y.shiraishi/Documents/github/solar-panel-segmentation/data/processed/20211128_thermal/solar/mask/' + os.path.split(folder)[-1] + '_' + file_name + f"_{cnt}_org.png", gray_img)
                            cv2.imwrite('/Users/y.shiraishi/Documents/github/solar-panel-segmentation/data/processed/20211128_thermal/solar/mask/' + os.path.split(folder)[-1] + '_' + file_name + f"_{cnt}_mask.png", double_mask)
                        else:
                            arr_name = os.path.split(folder)[-1] + '_' + file_name + f"_{cnt}"
                            np.save('/Users/y.shiraishi/Documents/github/solar-panel-segmentation/data/processed/20211128_thermal/empty/mask/' + arr_name, np.zeros_like(mask_arr))
                            np.save('/Users/y.shiraishi/Documents/github/solar-panel-segmentation/data/processed/20211128_thermal/empty/org/' + arr_name, 
                                    np.concatenate([np.expand_dims(gray_img, 0), np.expand_dims(gray_img, 0), np.expand_dims(gray_img, 0)]))

                        cnt += 1

                        print(cnt)

In [ ]:
print('END!!')

END!!


# ファイル調整

In [ ]:
c = 0
for item in glob.glob('/Users/y.shiraishi/Documents/github/solar-panel-segmentation/data/processed/20211128_thermal/solar/org/*.npy'):
    i = item.replace('/org/', '/mask/')
    
    if not(os.path.exists(i)):
        print(item)
        c += 1
        os.remove(item)
print(c)

In [ ]:
item

'/Users/y.shiraishi/Documents/github/solar-panel-segmentation/data/processed/20211128_thermal/solar/org/七飯峠下太陽光発電所_2021-09-17 13_32_34_サーモ_DJI_0027.JPG_1.npy'

In [ ]:
mask_arr.shape

(224, 224)

In [ ]:
for item in glob.glob('/Users/y.shiraishi/Documents/github/solar-panel-segmentation/data/processed/20211128_thermal/empty/mask/*.png'):
    png_name = os.path.split(item)[-1]
    name = os.path.splitext(png_name)[0]
    
    np.save('/Users/y.shiraishi/Documents/github/solar-panel-segmentation/data/processed/thermal/empty/mask/' + name, np.zeros_like(mask_arr))